In [27]:
from __future__ import print_function # Python 2/3 compatibility
import boto3
from botocore.exceptions import ClientError
import json
import decimal

# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

dynamodb = boto3.resource('dynamodb', region_name='eu-central-1')

In [28]:
def scan_table(table_name, filter_key=None, filter_value=None):
    """
    Perform a scan operation on table.
    Can specify filter_key (col name) and its value to be filtered.
    """
    table = dynamodb.Table(table_name)

    if filter_key and filter_value:
        filtering_exp = Key(filter_key).eq(filter_value)
        response = table.scan(FilterExpression=filtering_exp)
    else:
        response = table.scan()

    return response


In [31]:
def get_table_metadata(table_name):
    """
    Get some metadata about chosen table.
    """
    table = dynamodb.Table(table_name)

    return {
        'num_items': table.item_count,
        'primary_key_name': table.key_schema[0],
        'status': table.table_status,
        'bytes_size': table.table_size_bytes,
        'global_secondary_indices': table.global_secondary_indexes
    }

In [32]:
def read_table_item(table_name, pk_name, pk_value):
    """
    Return item read by primary key.
    """
    table = dynamodb.Table(table_name)
    response = table.get_item(Key={pk_name: pk_value})

    return response


def add_item(table_name, col_dict):
    """
    Add one item (row) to table. col_dict is a dictionary {col_name: value}.
    """
    table = dynamodb.Table(table_name)
    response = table.put_item(Item=col_dict)

    return response


def delete_item(table_name, pk_name, pk_value):
    """
    Delete an item (row) in table from its primary key.
    """
    table = dynamodb.Table(table_name)
    response = table.delete_item(Key={pk_name: pk_value})

    return

In [33]:
def query_table(table_name, filter_key=None, filter_value=None):
    """
    Perform a query operation on the table. 
    Can specify filter_key (col name) and its value to be filtered.
    """
    table = dynamodb_resource.Table(table_name)

    if filter_key and filter_value:
        filtering_exp = Key(filter_key).eq(filter_value)
        response = table.query(KeyConditionExpression=filtering_exp)
    else:
        response = table.query()

    return response

In [55]:
def scan_table_allpages(table_name, filter_key=None, filter_value=None):
    """
    Perform a scan operation on table. 
    Can specify filter_key (col name) and its value to be filtered. 
    This gets all pages of results. Returns list of items.
    """
    table = dynamodb.Table(table_name)

    if filter_key and filter_value:
        filtering_exp = Key(filter_key).eq(filter_value)
        response = table.scan(FilterExpression=filtering_exp)
    else:
        response = table.scan()

    items = response['Items']
    while True:
        print(len(response['Items']))
        if response.get('LastEvaluatedKey'):
            response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
            items += response['Items']
        else:
            break

    return items

In [49]:
print(len(parts['Items']))
for p in parts['Items'][0:10]:
    print(p['id'])
    #delete_item('parts','id',p['id'])

1103
LQW15AN4N6C80D
L-14CR12KV4T
SPM6550T-R68M
MHQ0402PSA15NHT000
ATFC-0402-3N0-BT
MHQ1005P3N6ST000
2200HT-680-H-RC
1227AS-H-120M=P2
EST1060T1R0NDGA
CLF10060NIT-2R2N-D


In [57]:
def batch_delete_all_data(table_name, filter_key=None, filter_value=None):
    table = dynamodb.Table(table_name)
    parts = scan_table_allpages(table_name)
    with table.batch_writer() as batch:
        for p in parts['Items']:
            pk_name = 'id'
            pk_value= p['id']
            batch.delete_item(Key={pk_name: pk_value})

In [ ]:
batch_delete_all_data('parts')

1102
1103
1099
1106
1104
1102
1102
1100
1101
1100
1100
1105
